# DALY Disability Weights sheets export to redshift

The *gestão de saúde magenta* squad inputs dw weights for daly calculations manually in a google sheet. This notebook is to export the sheet, after downloading from the Google Drive and storing it in a local computer. Since the sheet is updated not very often, there is not much gain from automating the process in Airflow (*yet*).

Steps:

    - Download the google sheet DALY weights and store it in the 'redshift_import_export\external\' package folder;

    - run this notebook to upload to Redshift;
    
    - use the DW to make calculations, new tables, dashes...
***

## 0. Setup

In [1]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [2]:
df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/ALICE CIDS-DALYs - Volumetria.csv')

In [3]:
df.columns

Index(['CID', 'subcategoria', 'categoria', 'severidade',
       'GHE Sequela/GBD Higher level', 'GHE/GBD 2019 health state',
       'GHE/GBD 2019 description', 'Peso DALY-Alice - limite inferior',
       'Peso DALY-Alice', 'Peso DALY-Alice - limite superior',
       'VALIDADO POR EQUIPE?', 'References', 'Notes', 'Unnamed: 13',
       'Unnamed: 14', 'EXTRAS'],
      dtype='object')

## 2. Transform

In [4]:
df_skew = df[['CID','severidade', 'Peso DALY-Alice - limite inferior','Peso DALY-Alice', 'Peso DALY-Alice - limite superior']]
df_skew.rename(columns ={'CID': 'cid',
                         'severidade': 'severity',
                         'Peso DALY-Alice - limite inferior': 'daly_weight_lower',
                         'Peso DALY-Alice': 'daly_weight',
                         'Peso DALY-Alice - limite superior': 'daly_weight_upper'}, inplace=True)

df_skew['cid2'] = df_skew['cid'].shift(1)
df_skew['cid3'] = np.where(df_skew['cid'].isna(), df_skew['cid2'], df_skew['cid'])
df_skew.drop(columns = ['cid', 'cid2'], inplace=True)
df_skew.rename(columns = {'cid3': 'cid'}, inplace=True)

## 3. Export

In [5]:
di.redshift.insert_table(df_skew, table_name='daly_weights_for_alice', if_exists='replace')

Table daly_weights_for_alice inserted successfully, with 480 rows.
